In [1]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |


import sys
!{sys.executable} -m pip install gpustat

In [3]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

#Extra Headers:
import os as os
import pywt as py
import statistics as st
import os as os
import random
import multiprocessing
from joblib import Parallel, delayed
import platform
import random


from time import time as ti

import CoreFunctions as cf
#from skimage.restoration import denoise_wavelet

import os
import pickle

# currently running pid 4010813

HostName = platform.node()

location = '/sciclone/home/dchendrickson01/image/'
rootfolder = '/sciclone/home/dchendrickson01/'
folder = '/scratch/RecordingsSplit/xFold/'

def Openfile(file):
    try:
        ff = open(folder+file,'rb')
        dump = pickle.load(ff)
    
        return dump[0], dump[1]
    except:
        print("bad file ",file)

In [4]:
location = folder
Titles = True
Ledgends = True

FileBatch = 20000

TimeSteps = 350
PredictSize = 25
Features = 3
MiddleLayerSize = 500

num_cores = 30
num_gpus = 2

files = os.listdir(folder)
print('files: ', len(files))

random.shuffle(files)

files:  678618


In [6]:
files[0][:-4]

'230510 recording1-0006'

In [2]:
#from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Masking, Lambda
from keras.models import Sequential
import tensorflow as tf

2024-06-17 15:04:08.348757: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 15:04:08.404965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
class LSTM_Autoencoder:
  def __init__(self, optimizer='adam', loss='mse'):
    self.optimizer = optimizer
    self.loss = loss
    self.n_features = Features
    self.timesteps = TimeSteps
    
  def build_model(self):
    timesteps = self.timesteps
    n_features = self.n_features
    model = Sequential()
    
    # Padding
    #model.add(Masking(mask_value=0.0, input_shape=(timesteps, n_features)))

    # Encoder
    model.add(LSTM(timesteps, activation='relu', input_shape=(TimeSteps, Features), return_sequences=True))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(LSTM(12, activation='relu'))
    model.add(RepeatVector(timesteps))
    
    # Decoder
    model.add(LSTM(timesteps, activation='relu', return_sequences=True))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(optimizer=self.optimizer, loss=self.loss, metrics=['accuracy'])
    model.summary()
    self.model = model
    
  def simple_model(self):
    
    # define model
    model = Sequential(name='DanModel')
    model.add(LSTM(MiddleLayerSize, input_shape=(TimeSteps * Features,1), return_sequences=True,name='danLSTM'))
    #model.add(RepeatVector(TimeSteps))
    #model.add(RepeatVector(PredictSize))
    
    #model.add(LSTM(25, return_sequences=True))
    
    model.add(TimeDistributed(Dense( MiddleLayerSize, activation='softmax',name='DanDense')))

    model.add(Lambda(lambda x: x[:, -PredictSize * Features:,1], name='DanLambda')) #Select last N from output  
    #https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras?noredirect=1&lq=1

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    self.model = model
    
  def fit(self, X, epochs=3, batch_size=32):
    #self.timesteps = np.shape(X)[0]
    #self.build_model()
    
    #input_X = np.expand_dims(X, axis=1)
    self.model.fit(X, X, epochs=epochs, batch_size=batch_size)
    
  def predict(self, X):
    #input_X = np.expand_dims(X, axis=1)
    output_X = self.model.predict(X)
    reconstruction = np.squeeze(output_X)
    return np.linalg.norm(X - reconstruction, axis=-1)
  
  def plot(self, scores, timeseries, threshold=0.95):
    sorted_scores = sorted(scores)
    threshold_score = sorted_scores[round(len(scores) * threshold)]
    
    plt.title("Reconstruction Error")
    plt.plot(scores)
    plt.plot([threshold_score]*len(scores), c='r')
    plt.show()
    
    anomalous = np.where(scores > threshold_score)
    normal = np.where(scores <= threshold_score)
    
    plt.title("Anomalies")
    plt.scatter(normal, timeseries[normal][:,-1], s=3)
    plt.scatter(anomalous, timeseries[anomalous][:,-1], s=5, c='r')
    plt.show()

lstm_autoencoder2 = LSTM_Autoencoder(optimizer='adam', loss='mse')

In [7]:
from tensorflow.python.keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                        inter_op_parallelism_threads=num_cores, 
                        allow_soft_placement=True,
                        device_count = {'CPU' : num_cores,
                                        'GPU' : num_gpus}
                       )
session = tf.compat.v1.Session(config=config)
K.set_session(session)

2024-06-17 12:06:30.994309: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22287 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:19:00.0, compute capability: 8.0
2024-06-17 12:06:30.998269: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22287 MB memory:  -> device: 1, name: NVIDIA A30, pci bus id: 0000:1c:00.0, compute capability: 8.0


In [8]:
lstm_autoencoder2.build_model()

2024-06-17 12:06:31.070388: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22287 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:19:00.0, compute capability: 8.0
2024-06-17 12:06:31.072496: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22287 MB memory:  -> device: 1, name: NVIDIA A30, pci bus id: 0000:1c:00.0, compute capability: 8.0
/sciclone/home/dchendrickson01/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 350, 350)       │       495,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 350, 50)        │        80,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 12)             │         3,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 350, 12)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 350, 350)       │       508,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 350, 50)        │        80,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 350, 3)         │           153 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,167,377 (4.45 MB)

 Trainable params: 1,167,377 (4.45 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
Datas = Openfile(files[3])

In [13]:
np.shape(Datas[0])

(93, 750, 3)

In [6]:
TotalMoves=0
Loops=int(len(files)/FileBatch)+1


random.shuffle(files)

k=0
print("Starting Loop "+str(k+1)+" of "+str(Loops+1))

start = k * FileBatch

Results = Parallel(n_jobs=2)(delayed(Openfile)(file) for file in files[start:start+FileBatch])

Moves = []
Names = []
for result in Results:
    try:
        for j in range(len(result[0])):
            Moves.append(result[0][j,:,:])
            Names.append(result[1]+str(i).zfill(5))
            i+=1
    except:
        pass

del Results

Starting Loop 1 of 35


/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


bad file  230111 recording4-002243.p
bad file  230210 recording1-001099.p


In [7]:
len(Moves)

19194

In [ ]:
X, y = list(), list()
for move in Moves:
    X.append(move[:TimeSteps,:].flatten())
    y.append(move[TimeSteps:TimeSteps+PredictSize,:].flatten())
    X.append(move[TimeSteps+PredictSize:2*TimeSteps+PredictSize,:].flatten())
    y.append(move[2*TimeSteps+PredictSize:,:].flatten())
    TotalMoves+=1

Batches = 32

with tf.device('/cpu:0'):
    X = tf.convert_to_tensor(X, np.float32)
    y = tf.convert_to_tensor(y, np.float32)

lstm_autoencoder2.model.fit(X, y, epochs=4, batch_size=Batches, verbose=2)

lstm_autoencoder2.model.save("LSTM_AtOnce_350p25")

print('Total Moves ',TotalMoves)

del X, y, Moves, Names

In [2]:
import numpy as np

In [42]:
Test = [1,2,3,4,5,6,7,8,9]

In [43]:
Test.type()

AttributeError: 'list' object has no attribute 'type'

In [11]:
Test.extend(np.zeros(5))
Test

[1, 2, 3, 4, 5, 6, 7, 8, 9, 0.0, 0.0, 0.0, 0.0, 0.0]

In [14]:
test = np.matrix(Test)

In [33]:
Test.insert(0, Test.pop())

In [34]:
test = np.concatenate((np.matrix(Test),test))

In [35]:
test

matrix([[0., 0., 0., 0., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 0., 0., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 0.],
        [0., 0., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 0., 0.],
        [0., 0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 0., 0., 0.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 0., 0., 0., 0.],
        [1., 2., 3., 4., 5., 6., 7., 8., 9., 0., 0., 0., 0., 0.]])

In [36]:
sVect = test.sum(axis=0)

In [37]:
sVect

matrix([[ 1.,  3.,  6., 10., 15., 21., 27., 33., 39., 35., 30., 24., 17.,
          9.]])

In [38]:
eVect = (test!=0).sum(axis=0)
eVect

matrix([[1, 2, 3, 4, 5, 6, 6, 6, 6, 5, 4, 3, 2, 1]])

In [39]:
VarVect = sVect / eVect

In [40]:
StdDev = np.sqrt(VarVect)

In [41]:
StdDev

matrix([[1.        , 1.22474487, 1.41421356, 1.58113883, 1.73205081,
         1.87082869, 2.12132034, 2.34520788, 2.54950976, 2.64575131,
         2.73861279, 2.82842712, 2.91547595, 3.        ]])

In [45]:
StdDev = np.asarray(StdDev)

In [46]:
np.append(StdDev,[0])

array([1.        , 1.22474487, 1.41421356, 1.58113883, 1.73205081,
       1.87082869, 2.12132034, 2.34520788, 2.54950976, 2.64575131,
       2.73861279, 2.82842712, 2.91547595, 3.        , 0.        ])

In [49]:
np.average(test,axis=0)

matrix([[0.16666667, 0.5       , 1.        , 1.66666667, 2.5       ,
         3.5       , 4.5       , 5.5       , 6.5       , 5.83333333,
         5.        , 4.        , 2.83333333, 1.5       ]])

In [1]:
import os
import shutil

In [2]:
Folder = '/scratch/1000Input/'

In [ ]:
%%time
file_list = [
    os.path.join(Folder,file)
    for file in os.listdir(Folder) if file.endswith('Data.csv') and file.startswith('2')
]

In [1]:
import glob

In [7]:
%%time
file_list = glob.glob(os.path.join(Folder, '*Data.csv'))

CPU times: user 1min 3s, sys: 3min 50s, total: 4min 53s
Wall time: 1h 32min 51s


In [8]:
len(file_list)


31584432

In [3]:
def list_csv_files(directory):
    csv_files = []
    with os.scandir(directory) as entries:
        for entry in entries:
            if entry.is_file() and entry.name.endswith('Data.csv'):
                csv_files.append(entry.path)
    return csv_files

In [ ]:
%%time
csv_files = list_csv_files(Folder)

In [ ]:
len(csv_files)


In [ ]:
from pathlib import Path

In [ ]:
[str(p) for p in Path(directory).iterdir() if p.is_file() and ]

In [4]:
import random

In [5]:
random.shuffle(file_list)

In [8]:
for i in range(2000):
    shutil.copy(file_list[i],'/scratch/1000Sm/')
    shutil.copy(file_list[i][:-8]+'Outs.csv','/scratch/1000Sm/')